# Introduction

Insomnia is one of the sleeping disorder. Although we may think that insomnia can occur only on adults, in fact it also can occur on adolescents. This data science project is delving into the insomnia during the adolescent period by using a dataset which contains the records of more than 90 adolecents. Individually, all participants have been following and participating in 19 questionnaires related to the sleep pattern, sleep habit, psychological issues, etc.

In [178]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

df = pd.read_csv(r"../datasets/insomnia_data.csv")


# Stage 1: Data Cleaning

Based on the observations toward the dataset, it requires some data cleaning steps before the dataset is processed on the next stages. On the following code, we can see the dimension and the first five rows of the dataset.

In [179]:
df.shape

(95, 174)

In [180]:
df.head()

,ID,Group,SubGroup,Remote,Sex,Age,American_Indian,Asian,Native_Hawaiian,Black,...,Zcope_acccept,Zcope_suppression,Zcope_planning,Zders_nonaccpetance,Zders_goals,Zders_impulse,Zders_awareness,Zders_strategies,Zders_clarity,ZDERS_total
0,sub_001,0,0,0,0,19.3,0,0,0,0,...,-0.701562,2.215354,0.740137,-1.122805,1.905727,-0.114565,1.083087,-0.656051,0.538016,0.575806
1,sub_002,0,0,0,0,19.3,0,0,0,0,...,1.545013,0.327833,1.078180,-0.552396,-0.302127,-0.114565,1.083087,-0.656051,0.538016,0.153943
2,sub_003,0,0,0,1,18.8,0,0,0,0,...,-0.327133,1.743474,-0.612036,-0.267191,-1.406055,-0.425527,0.271626,-0.656051,-0.260601,-0.619473
3,sub_004,0,0,0,0,18.8,0,0,0,0,...,-1.824849,-1.559689,-0.612036,-0.552396,0.249836,-0.114565,0.596210,-0.116442,0.538016,0.224254
4,sub_005,1,2,0,1,19.6,0,0,0,0,...,-2.199279,-0.615928,-0.950080,0.588422,1.353763,0.196397,0.109334,0.153363,1.336632,0.857049


## Drop non-total features from 'Multiple Columns with Total' questionnaire type

In [181]:
# Create seven lists which contain the non-total features name that will be dropped
non_total_ASHS = ['ASHS_physiological', 'ASHS_cognitive', 'ASHS_emotional', 'ASHS_SleepEnvirnmont', 'ASHS_DaytimeSleep', 'ASHS_substances', 'ASHS_bedtimeRoutine', 'ASHS_sleepStability', 'ASHS_BedroomSharing']
non_total_GCTI = ['GCTI_anxiety', 'GCTI_reflection', 'GCTI_worries', 'GCTI_thoughts', 'GCTI_negativeAffect']
non_total_PSRS = ['PSRS_PrR', 'PSRS_RWO', 'PSRS_RSC', 'PSRS_FRa', 'PSRS_RSE']
non_total_TCQIR = ['TCQIR_Aggressive_supression', 'TCQIR_cognitive_distraction', 'TCQIR_reappraisal', 'TCQIR_behavtioral_distraction', 'TCQIR_social_avoidance', 'TCQIR_worry']
non_total_ASQ = ['asq_home', 'asq_school', 'asq_attendance', 'asq_romantic', 'asq_peer', 'asq_teacher', 'asq_future', 'asq_leisure', 'asq_finance', 'asq_responsibility']
non_total_CASQ = ['casq_sleepy', 'casq_alert']
non_total_DERS = ['ders_nonaccpetance', 'ders_goals', 'ders_impulse', 'ders_awareness', 'ders_strategies', 'ders_clarity']

# Concatenate seven lists onto a new list
non_total_cols = np.concatenate((non_total_ASHS, 
                                 non_total_GCTI, 
                                 non_total_PSRS,
                                 non_total_TCQIR,
                                 non_total_ASQ,
                                 non_total_CASQ,
                                 non_total_DERS
                                 ))
print('Amount of non-total features: {}'.format(len(non_total_cols)))
print(non_total_cols)

Amount of non-total features: 43
['ASHS_physiological' 'ASHS_cognitive' 'ASHS_emotional'
 'ASHS_SleepEnvirnmont' 'ASHS_DaytimeSleep' 'ASHS_substances'
 'ASHS_bedtimeRoutine' 'ASHS_sleepStability' 'ASHS_BedroomSharing'
 'GCTI_anxiety' 'GCTI_reflection' 'GCTI_worries' 'GCTI_thoughts'
 'GCTI_negativeAffect' 'PSRS_PrR' 'PSRS_RWO' 'PSRS_RSC' 'PSRS_FRa'
 'PSRS_RSE' 'TCQIR_Aggressive_supression' 'TCQIR_cognitive_distraction'
 'TCQIR_reappraisal' 'TCQIR_behavtioral_distraction'
 'TCQIR_social_avoidance' 'TCQIR_worry' 'asq_home' 'asq_school'
 'asq_attendance' 'asq_romantic' 'asq_peer' 'asq_teacher' 'asq_future'
 'asq_leisure' 'asq_finance' 'asq_responsibility' 'casq_sleepy'
 'casq_alert' 'ders_nonaccpetance' 'ders_goals' 'ders_impulse'
 'ders_awareness' 'ders_strategies' 'ders_clarity']


In [182]:
# Drop non-total features from the dataframe
df.drop(non_total_cols, inplace=True, axis=1)

# Current dataframe dimension after non-total features dropping
print('Current dataframe dimension after non-total features dropping: {}'.format(df.shape))

Current dataframe dimension after non-total features dropping: (95, 131)


## Drop all features start with 'Z'

In [183]:
# Get all features name
cols_name = df.columns
# Initiate 'Z' letter
first_letter = 'Z'
# Get features name start with 'Z'
z_cols = [name for name in cols_name if name[0] == first_letter]
print("Amount of features name start with 'Z': {}".format(len(z_cols)))
print(z_cols)

Amount of features name start with 'Z': 78
['ZISI_total', 'ZPSQI_total', 'ZBDI_total', 'ZASHS_total', 'ZASHS_physiological', 'ZASHS_cognitive', 'ZASHS_emotional', 'ZASHS_SleepEnvirnmont', 'ZASHS_DaytimeSleep', 'ZASHS_substances', 'ZASHS_bedtimeRoutine', 'ZASHS_sleepStability', 'ZASHS_BedroomSharing', 'ZDBAS_total', 'ZFIRST_total', 'ZGCTI_total', 'ZGCTI_anxiety', 'ZGCTI_reflection', 'ZGCTI_worries', 'ZGCTI_thoughts', 'ZGCTI_negativeAffect', 'ZSTAI_Y_total', 'ZNEO_neuroticism', 'ZNEO_extraversion', 'ZNEO_openness', 'ZNEO_agreeableness', 'ZNEO_Conscientiousness', 'ZMEQr_total', 'ZPSRS_PrR', 'ZPSRS_RWO', 'ZPSRS_RSC', 'ZPSRS_FRa', 'ZPSRS_RSE', 'ZPSRS_total', 'ZPSS_total', 'ZTCQI_R_Total', 'ZTCQIR_Aggressive_supression', 'ZTCQIR_cognitive_distraction', 'ZTCQIR_reappraisal', 'ZTCQIR_behavtioral_distraction', 'ZTCQIR_social_avoidance', 'ZTCQIR_worry', 'ZACE_tot', 'Zasq_home', 'Zasq_school', 'Zasq_attendance', 'Zasq_romantic', 'Zasq_peer', 'Zasq_teacher', 'Zasq_future', 'Zasq_leisure', 'Zasq_fi

In [184]:
# Drop features name start with 'Z'
df.drop(z_cols, inplace=True, axis=1)

# Current dataframe dimension after dropping features name start with 'Z'
print("Current dataframe dimension after dropping features name start with 'Z': {}".format(df.shape))

Current dataframe dimension after dropping features name start with 'Z': (95, 53)


In [185]:
df.head()

,ID,Group,SubGroup,Remote,Sex,Age,American_Indian,Asian,Native_Hawaiian,Black,...,cope_religion,cope_humor,cope_disengage_emo,cope_restraint,cope_socialsupp_emo,cope_acccept,cope_suppression,cope_planning,ders_total,Unnamed: 95
0,sub_001,0,0,0,0,19.3,0,0,0,0,...,6,9,4,14,14,9,13,13,87,NaN
1,sub_002,0,0,0,0,19.3,0,0,0,0,...,5,5,4,8,8,15,9,14,81,NaN
2,sub_003,0,0,0,1,18.8,0,0,0,0,...,4,9,6,6,16,10,12,9,70,NaN
3,sub_004,0,0,0,0,18.8,0,0,0,0,...,4,8,4,7,13,6,5,9,82,NaN
4,sub_005,1,2,0,1,19.6,0,0,0,0,...,4,11,4,6,9,5,7,8,91,NaN


## Drop unnamed features

In [186]:
# Get unnamed features
unnamed_cols = [name for name in cols_name if "Unnamed" in name]
print("Amount of unnamed features: {}".format(len(unnamed_cols)))
print(unnamed_cols)

Amount of unnamed features: 1
['Unnamed: 95']


In [187]:
# Drop all unnamed features
df.drop(unnamed_cols, axis=1, inplace=True)
df.head(5)

,ID,Group,SubGroup,Remote,Sex,Age,American_Indian,Asian,Native_Hawaiian,Black,...,cope_denial,cope_religion,cope_humor,cope_disengage_emo,cope_restraint,cope_socialsupp_emo,cope_acccept,cope_suppression,cope_planning,ders_total
0,sub_001,0,0,0,0,19.3,0,0,0,0,...,4,6,9,4,14,14,9,13,13,87
1,sub_002,0,0,0,0,19.3,0,0,0,0,...,5,5,5,4,8,8,15,9,14,81
2,sub_003,0,0,0,1,18.8,0,0,0,0,...,4,4,9,6,6,16,10,12,9,70
3,sub_004,0,0,0,0,18.8,0,0,0,0,...,4,4,8,4,7,13,6,5,9,82
4,sub_005,1,2,0,1,19.6,0,0,0,0,...,4,4,11,4,6,9,5,7,8,91


## Combine 'PDS_FEMALE' and 'PDS_MALE' features onto a new single feature

In [188]:
# df.drop() code to prevent 'ValueError: cannot insert PDS_score, already exists' error message
df = df.drop(columns=['PDS_score'], errors='ignore')

# Insert new 'PDS_score' feature after 'PDS_MALE' feature
df.insert(df.columns.get_loc('PDS_MALE') + 1, 
          'PDS_score', 
          df.apply(lambda row: row['PDS_MALE'] if row['PDS_MALE'] > 0 else (row['PDS_FEMALE'] if row['PDS_FEMALE'] > 0 else None), axis=1))

# Drop 'PDS_FEMALE' and 'PDS_MALE' features
df = df.drop(columns=['PDS_FEMALE', 'PDS_MALE'], errors='ignore')

# Display the first feature until 'PDS_score' feature
df_partition = df.loc[:, :'PDS_score']
df_partition.head(5)

,ID,Group,SubGroup,Remote,Sex,Age,American_Indian,Asian,Native_Hawaiian,Black,White,unknown_Race,Hispanic,NotHispanic,unknown_Etnicity,PDS_score
0,sub_001,0,0,0,0,19.3,0,0,0,0,1,0,0,1,0,16.0
1,sub_002,0,0,0,0,19.3,0,0,0,0,1,0,0,1,0,13.0
2,sub_003,0,0,0,1,18.8,0,0,0,0,1,0,0,1,0,16.0
3,sub_004,0,0,0,0,18.8,0,0,0,0,1,0,0,1,0,17.0
4,sub_005,1,2,0,1,19.6,0,0,0,0,1,0,0,1,0,14.0


## Drop the duplicated records (if those are available)

In [189]:
# Check on the duplicated rows
duplicated_rows = df[df.duplicated()]
print("Amount of the duplicated records: {}".format(duplicated_rows.shape[0]))

Amount of the duplicated records: 0


## Drop records when 'age' is greater than 19

In [190]:
# Check on the records with 'age' value is greater than 19
adults = df.loc[df.Age >= 20.0]
print("Amount of the participants with age more than 19: {}".format(adults.shape[0]))
adults

Amount of the participants with age more than 19: 1


,ID,Group,SubGroup,Remote,Sex,Age,American_Indian,Asian,Native_Hawaiian,Black,...,cope_denial,cope_religion,cope_humor,cope_disengage_emo,cope_restraint,cope_socialsupp_emo,cope_acccept,cope_suppression,cope_planning,ders_total
7,sub_008,0,0,0,1,20.1,0,0,0,0,...,4,6,4,4,7,16,6,5,8,74


In [191]:
print("Original amount of the participants: {}".format(df.shape[0]))

# Drop the record of participant with age is more than 19
df = df.drop(df[df['Age'] >= 20.0].index)

print("Current amount of the participants: {}".format(df.shape[0]))

Original amount of the participants: 95
Current amount of the participants: 94


## Rename features

In [192]:
# Get the features name - before renaming process
old_cols_name = df.columns
old_cols_name

Index(['ID', 'Group', 'SubGroup', 'Remote', 'Sex', 'Age', 'American_Indian',
       'Asian', 'Native_Hawaiian', 'Black', 'White', 'unknown_Race',
       'Hispanic', 'NotHispanic', 'unknown_Etnicity', 'PDS_score', 'ISI_total',
       'PSQI_total', 'BDI_total', 'ASHS_total', 'DBAS_total', 'FIRST_total',
       'GCTI_total', 'STAI_Y_total', 'NEO_neuroticism', 'NEO_extraversion',
       'NEO_openness', 'NEO_agreeableness', 'NEO_Conscientiousness',
       'MEQr_total', 'PSRS_total', 'PSS_total', 'TCQI_R_Total', 'ACE_tot',
       'casq_total', 'cope_disengage_su', 'cope_growth',
       'cope_disengage_mental', 'cope_emotions', 'cope_socialsupp_instr',
       'cope_active', 'cope_denial', 'cope_religion', 'cope_humor',
       'cope_disengage_emo', 'cope_restraint', 'cope_socialsupp_emo',
       'cope_acccept', 'cope_suppression', 'cope_planning', 'ders_total'],
      dtype='object')

In [193]:
# Create dictionary which contains the old and new features name
cols_new_name = {'unknown_Race': 'Unknown_Race',
                 'NotHispanic': 'Not_Hispanic',
                 'unknown_Etnicity': 'Unknown_Etnicity',
                 'PDS_score': 'PDS_Score',
                 'ISI_total': 'ISI_Total',
                 'PSQI_total': 'PSQI_Total',
                 'BDI_total': 'BDI_Total',
                 'ASHS_total': 'ASHS_Total',
                 'DBAS_total': 'DBAS_Total',
                 'FIRST_total': 'FIRST_Total',
                 'GCTI_total': 'GCTI_Total',
                 'STAI_Y_total': 'STAI_Y_Total',
                 'NEO_neuroticism': 'NEO_Neuroticism',
                 'NEO_extraversion': 'NEO_Extraversion',
                 'NEO_openness': 'NEO_Openness',
                 'NEO_agreeableness': 'NEO_Agreeableness',
                 'MEQr_total': 'MEQr_Total',
                 'PSRS_total': 'PSRS_Total',
                 'PSS_total': 'PSS_Total',
                 'ACE_tot': 'ACE_Total',
                 'casq_total': 'CASQ_Total',
                 'cope_disengage_su': 'COPE_Substance_Use',
                 'cope_growth': 'COPE_Growth',
                 'cope_disengage_mental': 'COPE_Mental_Disengagement',
                 'cope_emotions': 'COPE_Emotions',
                 'cope_socialsupp_instr': 'COPE_Social_Support_Instrumental',
                 'cope_active': 'COPE_Active',
                 'cope_denial': 'COPE_Denial',
                 'cope_religion': 'COPE_Religion',
                 'cope_humor': 'COPE_Humor',
                 'cope_disengage_emo': 'COPE_Behavioral_Disengagement',
                 'cope_restraint': 'COPE_Restraint',
                 'cope_socialsupp_emo': 'COPE_Social_Support_Emotional',
                 'cope_acccept': 'COPE_Accceptance',
                 'cope_suppression': 'COPE_Suppression',
                 'cope_planning': 'COPE_Planning',
                 'ders_total': 'DERS_Total'        
}

# Rename the features based on a 'cols_new_name' dictionary
df.rename(columns=cols_new_name, inplace=True)

# Get the features name - after renaming process
current_cols_name = df.columns
current_cols_name

Index(['ID', 'Group', 'SubGroup', 'Remote', 'Sex', 'Age', 'American_Indian',
       'Asian', 'Native_Hawaiian', 'Black', 'White', 'Unknown_Race',
       'Hispanic', 'Not_Hispanic', 'Unknown_Etnicity', 'PDS_Score',
       'ISI_Total', 'PSQI_Total', 'BDI_Total', 'ASHS_Total', 'DBAS_Total',
       'FIRST_Total', 'GCTI_Total', 'STAI_Y_Total', 'NEO_Neuroticism',
       'NEO_Extraversion', 'NEO_Openness', 'NEO_Agreeableness',
       'NEO_Conscientiousness', 'MEQr_Total', 'PSRS_Total', 'PSS_Total',
       'TCQI_R_Total', 'ACE_Total', 'CASQ_Total', 'COPE_Substance_Use',
       'COPE_Growth', 'COPE_Mental_Disengagement', 'COPE_Emotions',
       'COPE_Social_Support_Instrumental', 'COPE_Active', 'COPE_Denial',
       'COPE_Religion', 'COPE_Humor', 'COPE_Behavioral_Disengagement',
       'COPE_Restraint', 'COPE_Social_Support_Emotional', 'COPE_Accceptance',
       'COPE_Suppression', 'COPE_Planning', 'DERS_Total'],
      dtype='object')

In [194]:
# Count the amount of the changed features name
cols_differences = list(set(current_cols_name) - set(old_cols_name))
print("Amount of changed features name: {}".format(len(cols_differences)))

Amount of changed features name: 37


## Check and change the incorrect features datatype

In [195]:
original_dtypes = df.dtypes.tolist()
original_dtypes

[dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64')]

In [196]:
object_cols = df.select_dtypes(include=['object'])
object_cols.columns

Index(['ID', 'DBAS_Total', 'COPE_Religion'], dtype='object')

In [197]:
# Check the availability of null data in the object type features
df[['DBAS_Total', 'COPE_Religion']].isnull().sum()

DBAS_Total       0
COPE_Religion    1
dtype: int64

In [198]:
# Change features with object datatype to float datatype by changing null data into NaN
df[['DBAS_Total', 'COPE_Religion']] = df[['DBAS_Total', 'COPE_Religion']].apply(pd.to_numeric, errors='coerce')

# Check the current availability of features with object datatype
current_object_cols = df.select_dtypes(include=['object'])
current_object_cols.columns

Index(['ID'], dtype='object')

## Count on missing data availability on Dataframe

In [199]:
# Check on amount of data with missing data availability
df.isnull().sum()

ID                                  0
Group                               0
SubGroup                            0
Remote                              0
Sex                                 0
Age                                 0
American_Indian                     0
Asian                               0
Native_Hawaiian                     0
Black                               0
White                               0
Unknown_Race                        0
Hispanic                            0
Not_Hispanic                        0
Unknown_Etnicity                    0
PDS_Score                           0
ISI_Total                           0
PSQI_Total                          0
BDI_Total                           0
ASHS_Total                          0
DBAS_Total                          1
FIRST_Total                         0
GCTI_Total                          0
STAI_Y_Total                        0
NEO_Neuroticism                     0
NEO_Extraversion                    0
NEO_Openness

In [200]:
# Get the features name with missing data availability
missing_data_cols = df.columns[df.isna().any()].tolist()
missing_data_cols

['DBAS_Total', 'ACE_Total', 'COPE_Substance_Use', 'COPE_Religion']

## Drop 'Group' features from the Dataframe

In [201]:
# Dataframe header before 'Group' feature dropping
df.head(5)

,ID,Group,SubGroup,Remote,Sex,Age,American_Indian,Asian,Native_Hawaiian,Black,...,COPE_Denial,COPE_Religion,COPE_Humor,COPE_Behavioral_Disengagement,COPE_Restraint,COPE_Social_Support_Emotional,COPE_Accceptance,COPE_Suppression,COPE_Planning,DERS_Total
0,sub_001,0,0,0,0,19.3,0,0,0,0,...,4,6.0,9,4,14,14,9,13,13,87
1,sub_002,0,0,0,0,19.3,0,0,0,0,...,5,5.0,5,4,8,8,15,9,14,81
2,sub_003,0,0,0,1,18.8,0,0,0,0,...,4,4.0,9,6,6,16,10,12,9,70
3,sub_004,0,0,0,0,18.8,0,0,0,0,...,4,4.0,8,4,7,13,6,5,9,82
4,sub_005,1,2,0,1,19.6,0,0,0,0,...,4,4.0,11,4,6,9,5,7,8,91


In [202]:
# Drop 'Group' features from the Dataframe
df = df.drop(columns=['Group'], axis=1)

# Dataframe header after 'Group' feature dropping
df.head(5)

,ID,SubGroup,Remote,Sex,Age,American_Indian,Asian,Native_Hawaiian,Black,White,...,COPE_Denial,COPE_Religion,COPE_Humor,COPE_Behavioral_Disengagement,COPE_Restraint,COPE_Social_Support_Emotional,COPE_Accceptance,COPE_Suppression,COPE_Planning,DERS_Total
0,sub_001,0,0,0,19.3,0,0,0,0,1,...,4,6.0,9,4,14,14,9,13,13,87
1,sub_002,0,0,0,19.3,0,0,0,0,1,...,5,5.0,5,4,8,8,15,9,14,81
2,sub_003,0,0,1,18.8,0,0,0,0,1,...,4,4.0,9,6,6,16,10,12,9,70
3,sub_004,0,0,0,18.8,0,0,0,0,1,...,4,4.0,8,4,7,13,6,5,9,82
4,sub_005,2,0,1,19.6,0,0,0,0,1,...,4,4.0,11,4,6,9,5,7,8,91


## Create the flag features for the features with missing data

In [203]:
print("Amount of features before the flag features addition: {}".format(df.shape[1]))

Amount of features before the flag features addition: 50


In [204]:
# Create flag features in Dataframe
for column in df.columns[df.isna().any()]:
    df[f'{column}_flag'] = df[column].isna().astype(int)
    
print("Amount of features after the flag features addition: {}".format(df.shape[1]))
df.head(5)

Amount of features after the flag features addition: 54


,ID,SubGroup,Remote,Sex,Age,American_Indian,Asian,Native_Hawaiian,Black,White,...,COPE_Restraint,COPE_Social_Support_Emotional,COPE_Accceptance,COPE_Suppression,COPE_Planning,DERS_Total,DBAS_Total_flag,ACE_Total_flag,COPE_Substance_Use_flag,COPE_Religion_flag
0,sub_001,0,0,0,19.3,0,0,0,0,1,...,14,14,9,13,13,87,0,0,0,0
1,sub_002,0,0,0,19.3,0,0,0,0,1,...,8,8,15,9,14,81,0,0,0,0
2,sub_003,0,0,1,18.8,0,0,0,0,1,...,6,16,10,12,9,70,1,0,0,0
3,sub_004,0,0,0,18.8,0,0,0,0,1,...,7,13,6,5,9,82,0,0,0,0
4,sub_005,2,0,1,19.6,0,0,0,0,1,...,6,9,5,7,8,91,0,0,0,0


## Imputation for the missing data

In [205]:
# Define the imputer and the type of imputation - 'most frequent' equals to modus
imputer = SimpleImputer(strategy='most_frequent')

# Execute the imputation towards the feature with missing data
df[missing_data_cols] = imputer.fit_transform(df[missing_data_cols])

# Check the availability of missing data after the imputation
df[missing_data_cols].isna().sum()

# Convert Dataframe into CSV file
# df.to_csv('imputed_df.csv', sep=',', index=False, encoding='utf-8')    

DBAS_Total            0
ACE_Total             0
COPE_Substance_Use    0
COPE_Religion         0
dtype: int64

# Stage 2: Data Exploration

## Part 1: Feature Engineering

### Add average (mean) features for 'COPE' and 'NEO' questionnaires

In [206]:
# Get all COPE questionnaire features name
cope_columns_raw = [column for column in df.columns if 'COPE' in column]
cope_columns = [column for column in cope_columns_raw if 'flag' not in column]
print("All COPE features in Dataframe: {}".format(cope_columns))

All COPE features in Dataframe: ['COPE_Substance_Use', 'COPE_Growth', 'COPE_Mental_Disengagement', 'COPE_Emotions', 'COPE_Social_Support_Instrumental', 'COPE_Active', 'COPE_Denial', 'COPE_Religion', 'COPE_Humor', 'COPE_Behavioral_Disengagement', 'COPE_Restraint', 'COPE_Social_Support_Emotional', 'COPE_Accceptance', 'COPE_Suppression', 'COPE_Planning']


In [207]:
# Create the average (mean) feature of 'COPE' questionnaire
COPE_mean = df[cope_columns].mean(axis=1)
COPE_mean = COPE_mean.round(2)
# Place average (mean) feature of 'COPE' questionnaires after 'COPE_Planning feature position'
df.insert(loc = df.columns.get_loc('COPE_Planning') + 1, 
          column = 'COPE_mean', 
          value = COPE_mean)

In [208]:
df.loc[:, 'COPE_Substance_Use':'COPE_mean']

,COPE_Substance_Use,COPE_Growth,COPE_Mental_Disengagement,COPE_Emotions,COPE_Social_Support_Instrumental,COPE_Active,COPE_Denial,COPE_Religion,COPE_Humor,COPE_Behavioral_Disengagement,COPE_Restraint,COPE_Social_Support_Emotional,COPE_Accceptance,COPE_Suppression,COPE_Planning,COPE_mean
0,4.0,16,7,5,12,13,4,6.0,9,4,14,14,9,13,13,9.53
1,4.0,15,10,5,12,12,5,5.0,5,4,8,8,15,9,14,8.73
2,4.0,14,9,10,14,7,4,4.0,9,6,6,16,10,12,9,8.93
3,4.0,11,7,13,9,11,4,4.0,8,4,7,13,6,5,9,7.67
4,8.0,10,8,5,5,8,4,4.0,11,4,6,9,5,7,8,6.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,4.0,8,8,7,9,10,4,4.0,9,6,8,4,7,8,9,7.00
91,4.0,12,11,9,11,12,5,4.0,16,8,13,8,12,8,12,9.67
92,6.0,13,8,6,7,12,5,7.0,16,6,7,6,13,8,13,8.87
93,4.0,16,7,10,10,12,4,9.0,4,4,7,10,16,9,15,9.13


In [210]:
# Get all NEO questionnaire features name
neo_columns_raw = [column for column in df.columns if 'NEO' in column]
neo_columns = [column for column in neo_columns_raw if 'flag' not in column]
print("All NEO features in Dataframe: {}".format(neo_columns))

All NEO features in Dataframe: ['NEO_Neuroticism', 'NEO_Extraversion', 'NEO_Openness', 'NEO_Agreeableness', 'NEO_Conscientiousness']


In [211]:
# Create the average (mean) feature of 'COPE' questionnaire
NEO_mean = df[neo_columns].mean(axis=1)
NEO_mean = NEO_mean.round(2)
# Place average (mean) feature of 'COPE' questionnaires after 'COPE_Planning feature position'
df.insert(loc = df.columns.get_loc('NEO_Conscientiousness') + 1, 
          column = 'NEO_mean', 
          value = NEO_mean)

In [212]:
df.loc[:, 'NEO_Neuroticism':'NEO_mean']

,NEO_Neuroticism,NEO_Extraversion,NEO_Openness,NEO_Agreeableness,NEO_Conscientiousness,NEO_mean
0,12,30,25,18,33,23.6
1,9,23,24,18,30,20.8
2,19,27,29,21,33,25.8
3,15,33,22,18,27,23.0
4,15,28,28,33,34,27.6
...,...,...,...,...,...,...
90,18,20,28,27,32,25.0
91,25,37,29,31,23,29.0
92,17,28,17,33,29,24.8
93,4,39,28,43,46,32.0
